In [2]:
from models.processing import filter_dataset, load_dataset
from dataclasses import dataclass
from typing import List
import pandas as pd
from gurobipy import *

In [3]:
def load_dataset(data_input):
    """
    Load dataset from the preprocessed CSV
    :return:
    """
    df = pd.read_csv(data_input)
    return df

In [4]:
import random
def generate_distance_matrix(locations):
    """
    Given a list of n tuples representing location coordinates, return an nxn matrix containing all of the distances
    :param locations: list of n tuples representing location coordinates
    :return: nxn matrix containing all of the distances
    """
    dima=[[0 for i in range(len(locations))] for j in range(len(locations))]
    for i in range(len(locations)):
        for j in range(len(locations)):
            if i==j:
                dima[i][j]=0
            elif dima[j][i]!=0:
                dima[i][j]=dima[j][i]
            else:
                dima[i][j]=random.randint(1,101)/180
    return dima

In [5]:
@dataclass
class Bar:
    name: str
    latitude: str
    longitude: str
    rating: float

In [6]:
@dataclass
class Solution:
    bars = List[Bar]
    total_max_walking_time = float

In [76]:
def get_optimal_route(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each, max_total_wait):
    """

    :param df:
    :param start_time:
    :param end_time:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :return: A Solution class object representing the optimal solution
    """
    
    #parameters
    bigm=999999
    m=Model("opt_route")
    locations=df['name']
    ratings=df['stars']
    open_times = df['Friday open']
    close_times = df['Friday close']
    wait_times = df['wait_time']/60
    
    time_spent_each_bar = max(0.25, (end_time-start_time - total_max_walking_time - max_total_wait) / bar_num)
    
    y=[]
    x=[[0 for j in range(len(locations))] for i in range(len(locations))]
    z=[[[0 for j in range(len(locations))] for i in range(len(locations))] for k in range(bar_num-1)]
    #dima=generate_distance_matrix(locations)

    # create decision variables
    for loc in locations:
        y.append(m.addVar(vtype=GRB.BINARY, name="y_{}".format(loc)))
   
    for k in range(bar_num-1):
        for i in range(len(locations)):
            for j in range(len(locations)):
                z[k][i][j]=m.addVar(vtype=GRB.BINARY,name="z_{},{},{}".format(k,locations[i],locations[j]))
     
    ### objective function
    m.setObjective(quicksum([y[i]*ratings[i] for i in range(len(locations))]),GRB.MAXIMIZE)
    
    ### constraints
    # Number of locations visited
    m.addConstr(quicksum(y)==bar_num)
    
    # max total walk time
    m.addConstr(quicksum([z[w][i][j] * dima[i][j] 
                                           for i in range(len(locations))
                                           for j in range(len(locations))
                                           for w in range(bar_num-1)]) <= total_max_walking_time)
    
    # max walk time between locations
    for k in range(bar_num-1):
        m.addConstr(quicksum([z[k][i][j] * dima[i][j] 
                                           for i in range(len(locations))
                                           for j in range(len(locations))]) <= max_walking_each)
    
    # rules about z
    # no movements between the same bar
    for k in range(bar_num-1):
        for i in range(len(locations)):
            m.addConstr(z[k][i][i]==0)
            
    # froms/tos upper bound
    for i in range(len(locations)):
        m.addConstr(quicksum([z[k][i][j] for k in range(bar_num - 1) for j in range(len(locations))])
                    +quicksum([z[k][j][i] for k in range(bar_num - 1) for j in range (len(locations))])
                    <= bigm*y[i])
        
    # froms/tos lower bound
    for i in range(len(locations)):
        m.addConstr(quicksum([z[k][i][j] for k in range(bar_num - 1) for j in range(len(locations))])
                    +quicksum([z[k][j][i] for k in range(bar_num - 1) for j in range (len(locations))])
                    >= y[i]/bigm)
        
    # can only have one 1 per movement matrix
    for k in range(bar_num-1):
        m.addConstr(quicksum([z[k][i][j] for i in range(len(locations)) for j in range (len(locations))])==1)
    
    # make sure we don't vist the same bar twice
    #dimension1
    for i in range(len(locations)):
        m.addConstr(quicksum([z[k][i][j] for j in range(len(locations)) for k in range (bar_num-1)])<=1)
     
    #dimension2
    for j in range(len(locations)):
        m.addConstr(quicksum([z[k][i][j] for i in range(len(locations)) for k in range (bar_num-1)])<=1)
        
    # have to start from the bar you previously went to
    for k in range(1,bar_num-1):
        for i in range(len(locations)):
            m.addConstr(quicksum([z[k-1][j][i] 
                                  for j in range(len(locations))])==quicksum([z[k][i][j] for j in range(len(locations))]))
    
    #open  time - only distance is considered for the time being
    for zed in range(bar_num-1):
        m.addConstr(start_time + zed*time_spent_each_bar + quicksum([z[w][i][j] * (dima[i][j] + wait_times[i])
                                           for i in range(len(locations))
                                           for j in range(len(locations))
                                           for w in range(zed)]) >= quicksum([open_times[i]*quicksum(z[zed][i])
                                                                                for i in range(len(locations))]))
    m.addConstr(start_time + zed*time_spent_each_bar + quicksum([z[w][i][j] * (dima[i][j] + wait_times[i])
                                           for i in range(len(locations))
                                           for j in range(len(locations))
                                           for w in range(bar_num-1)]) >=
                                quicksum([open_times[j]*quicksum([z[bar_num-2][i][j] for i in range(len(locations))])
                                                                 for j in range(len(locations))]))
    # Close time constraint            
    for zed in range(bar_num-1):
        m.addConstr(start_time + zed*time_spent_each_bar + quicksum([z[w][i][j] * (dima[i][j] + wait_times[i])
                                           for i in range(len(locations))
                                           for j in range(len(locations))
                                           for w in range(zed)]) <= quicksum([close_times[i]*quicksum(z[zed][i])
                                                                                for i in range(len(locations))]))
    m.addConstr(start_time + zed*time_spent_each_bar + quicksum([z[w][i][j] * (dima[i][j] + wait_times[i]) 
                                           for i in range(len(locations))
                                           for j in range(len(locations))
                                           for w in range(bar_num-1)]) <=
                                quicksum([close_times[j]*quicksum([z[bar_num-2][i][j] for i in range(len(locations))])
                                                                 for j in range(len(locations))]))
    
    # Must exit last bar before close time
    m.addConstr(start_time + zed*time_spent_each_bar + quicksum([z[w][i][j] * (dima[i][j] + wait_times[i])
                                           for i in range(len(locations))
                                           for j in range(len(locations))
                                           for w in range(bar_num-1)]) <= end_time)

    # Total wait time less than max allowed
    m.addConstr(quicksum([wait_times[i]*y[i] for i in range(len(locations))]) <= max_total_wait)
    m.setParam('TimeLimit', 30)
    m.setParam('MIPFocus',1)
    m.optimize()
    return m,z,y,x,dima

In [71]:
def get_pareto_routes(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each):
    """
    Returns total_max_walking_time / 5 suggested routes (e.g. one for 0-5 mins walk, one for 5-10 mins walk, etc.).
    Pareto routes contain
    :param df:
    :param start_time:
    :param end_time:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :return: A list of Solutions
    """
    solutions = []
    for max_time in range(0, int(total_max_walking_time), 5):
        solutions.append(get_optimal_route(df, start_time, end_time, bar_num, max_time, max_walking_each))
    pass

In [79]:
def crawl_model(date, start_time, end_time, budget, bar_num,  total_max_walking_time,  max_walking_each, min_review_ct,
                min_review, city):
    """
    :param date:
    :param start_time:
    :param end_time:
    :param budget:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :param min_review_ct:
    :param min_review:
    :param city:
    :return:
    """

    df = load_dataset()
    df = filter_dataset(df, min_review_ct, min_review, date, budget, city)
    pareto_df = get_pareto_routes(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each)

    return pareto_df

In [80]:
toronto_data=load_dataset('data/full_example.csv')

In [89]:
toronto_data_filtered = toronto_data.loc[lambda f: ~f['Friday open'].isnull()].reset_index()
#wait_time=generate_distance_matrix
optrout,z,y,x,dima=get_optimal_route(toronto_data_filtered[:50], 17, 22, 6, 1.15, 0.25, 1)

Changed value of parameter TimeLimit to 30.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Optimize a model with 676 rows, 12550 columns and 243532 nonzeros
Variable types: 0 continuous, 12550 integer (12550 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+06]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 2e+01]
Presolve removed 288 rows and 8501 columns
Presolve time: 0.50s
Presolved: 388 rows, 4049 columns, 62310 nonzeros
Variable types: 0 continuous, 4049 integer (4048 binary)
Found heuristic solution: objective 20.5000000
Found heuristic solution: objective 22.0000000

Root relaxation: objective 2.850000e+01, 649 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      28.5000000  

In [95]:
total_dist = 0
total_wait = 0
wait_time = toronto_data['wait_time']
for k in range(len(z)):
    for i in range(len(z[0])):
        for j in range(len(z[0][0])):
            if z[k][i][j].x!=0:
                print(y[i].VarName)
                print(y[j].VarName)
print(total_dist)
print(total_wait/60)

y_Koerner Hall
y_Arthur Murray Dance Studio
y_Arthur Murray Dance Studio
y_I'll Be Seeing You
y_I'll Be Seeing You
y_Locus144
y_Locus144
y_The Wicket Bar
y_The Wicket Bar
y_Another Bar
0
0.0


In [98]:
optrout.objval

28.5

In [ ]:
total_dist = 0
total_wait = 0
wait_time = toronto_data['wait_time']
for k in range(6-1):
    for i in range(50):
        for j in range(50):
            if z[k][i][j].x!=0:
                print(z[k][i][j], k, i, j)
                total_dist += dima[i][j]
                total_wait += wait_time[i]
print(total_dist)
print(total_wait/60)